In [3]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=198
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792932/scfs'
TARGET_SPECIES='N'
####################################################

os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()




def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()





av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    eta0s.append(eta0)
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    
    
    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")


    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f1
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(os.getcwd()))
print("==================================")
print("""
target_atom_labels=\t{}\n
av_theta_xs=\t{}\n    
av_theta_ys=\t{}\n  
av_theta_y2s=\t{}\n
av_theta_x2s=\t{}\n 
cq_coef0s=\t{}\n
cq_coef1s=\t{}\n
eta_coefs=\t{}\n
cq0s=\t{}\n
cqprime0s=\t{}\n
cqprime1s=\t{}\n 
eta0s=\t{}\n
etaprimes=\t{}\n
fq0s=\t{}\n 
fqprimes=\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()


print("eta0s\n---------")
for item in target_atom_labels:
    print(item, eta0s[target_atom_labels.index(item)])
print()





endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()




Begin on Mon Feb 19 15:12:00 2018

Working dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792932/scfs


TEMP:            198
TARGET_DIR:      /Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792932/scfs
TARGET_SPECIES:  N


There are 901 efg files in this directory.

901 efg files opened without error.

The indices of N atoms are:
1 3 4 6 23 25 26 28 37 39 40 42 51 53 54 56 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
skipped:799
skipped:800
skipped:801
skipped:802
skipped:803
skipped:804
skipped:805
skipped:806
skipped:807
skipped:808
skipped:809
skipped:810
skipped:811
skipped:812
skipped:813
skipped:814
skipped:815
skipped:816
skipped:817
skipped:818
skipped:819
skipped:820
skipped:821
skipped:822
skipped:823
skipped:824
skipped:825
skipped:826
skipped:827
skipped:828
skipped:829
skipped:830
skipped:831
skipped:832
skipped:833
skipped:834
skipped:835
skipped:836
skipped:837
skipped:838
skipped:839
skipped:840
skipped:841
skipped:842
skipped:843
skipp

<tx>:	 -0.00539625358579 	<ty>:	 -0.0103643594574 
 <tx^2>	: 0.0296565925987 	<ty^2>: 0.0163656166949 

cq0=	-0.7865
eta0=	0.58515
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792932/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792932/scfs
T=198K, nuclear site: 6  =
1-3/2(<tx^2>+<ty^2>)		=0.930966686059635
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.9367995969722228

Cq0:		-0.7865
Cq' from coefficient 1-3/2(<>+<>):		-0.7322052985859029
Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):	-0.7367928830186532
Computing NQR frequencies for spin:	1
using function <function f1 at 0x11285fe18>	and simple coefficient
fqs=[-0.21422497 -0.44204149 -0.65626646]



Working on atomic specie:	23
Fetching the angles in file efg.{}.out
skipped:799
skipped:800
skipped:801
skipped:802
skipped:803
skipped:804
skipped:805
skipped:806
skipped:807
skipped:808
skipped:809
skipped:810
skipped:811
skipped:812
skipped:813
skipped:814
skipped:815
skipped:816
skipped:817
skipped:818
s

1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.9923667213256575

Cq0:		-6.2222
Cq' from coefficient 1-3/2(<>+<>):		-6.172183005419325
Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):	-6.1747042134325065
Computing NQR frequencies for spin:	1
using function <function f1 at 0x11285fe18>	and simple coefficient
fqs=[-1.56822826 -3.84502313 -5.41325138]



Working on atomic specie:	37
Fetching the angles in file efg.{}.out
skipped:799
skipped:800
skipped:801
skipped:802
skipped:803
skipped:804
skipped:805
skipped:806
skipped:807
skipped:808
skipped:809
skipped:810
skipped:811
skipped:812
skipped:813
skipped:814
skipped:815
skipped:816
skipped:817
skipped:818
skipped:819
skipped:820
skipped:821
skipped:822
skipped:823
skipped:824
skipped:825
skipped:826
skipped:827
skipped:828
skipped:829
skipped:830
skipped:831
skipped:832
skipped:833
skipped:834
skipped:835
skipped:836
skipped:837
skipped:838
skipped:839
skipped:840
skipped:841
skipped:842
skipped:843
skipped:844
skipped:845
skipped:846
sk

1-3/2(<tx^2>+<ty^2>)		=0.9914866230236895
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.9919437434695106

Cq0:		-6.2222
Cq' from coefficient 1-3/2(<>+<>):		-6.169228065778001
Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):	-6.172072360615989
Computing NQR frequencies for spin:	1
using function <function f1 at 0x11285fe18>	and simple coefficient
fqs=[-1.56747747 -3.84318232 -5.41065978]



Working on atomic specie:	51
Fetching the angles in file efg.{}.out
skipped:799
skipped:800
skipped:801
skipped:802
skipped:803
skipped:804
skipped:805
skipped:806
skipped:807
skipped:808
skipped:809
skipped:810
skipped:811
skipped:812
skipped:813
skipped:814
skipped:815
skipped:816
skipped:817
skipped:818
skipped:819
skipped:820
skipped:821
skipped:822
skipped:823
skipped:824
skipped:825
skipped:826
skipped:827
skipped:828
skipped:829
skipped:830
skipped:831
skipped:832
skipped:833
skipped:834
skipped:835
skipped:836
skipped:837
skipped:838
skipped:839
skipped:840
skipped:841
skipped:842
skipped:8

In [4]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime


target_atom_labels=	[1, 3, 4, 6, 23, 25, 26, 28, 37, 39, 40, 42, 51, 53, 54, 56]

av_theta_xs=	[-0.0080132048054610151, -0.0046117945404929445, 0.01293970448814552, -0.0053962535857870831, -0.03149234894724675, 0.0064250205945930077, 0.0056226886184239161, 0.012346513267368218, -0.0079666557682747351, 0.021704680018767911, -0.003263621608675109, -0.014095640524121487, -0.036356105463644234, 0.0034408609390223039, 0.010317133297643366, 0.016046205242009073]
    
av_theta_ys=	[0.0032100817271020353, 0.0046922255115280304, -0.012889789823419262, -0.010364359457355183, 0.0015939957025948666, -0.00026560022509702907, -0.011807942261037367, 0.0030022387384275482, -0.00037912141518146614, -0.0094345769335577815, -0.021158574379233411, -0.0011926239780507835, 0.0013293887622034834, -0.0018710252325972987, -0.0050004224886199006, 0.004964580922299627]
  
av_theta_y2s=	[0.00221338279719139, 0.018671061589008473, 0.0023381304902584435, 0.01636561669488101, 0.0086722681360291431, 0.0021663286622427939, 0.017593414546617774, 0.0021479061050531842, 0.014154261233330965, 0.001628543126326364, 0.019502891478626029, 0.0022380856112509022, 0.013217741130708742, 0.0022807472872109695, 0.01279460263418262, 0.0021275972923078704]

av_theta_x2s=	[0.0024252987370314331, 0.041329439630954742, 0.0024985951731757933, 0.029656592598695714, 0.029507992729934748, 0.0039874946798806876, 0.013352478978711798, 0.0032110767392435433, 0.038911394833967987, 0.0033867262400780291, 0.018654877685458749, 0.0034374990396227449, 0.033387349802887781, 0.0033404850648079158, 0.018304188686725127, 0.0027509854356645379]
 
cq_coef0s=	[0.99304197769866576, 0.90999924817005517, 0.99274491150484867, 0.93096668605963495, 0.94272960870105416, 0.99076926498681472, 0.95358115971200563, 0.99196152573355489, 0.92040151589905161, 0.99247709595039346, 0.94276334625387281, 0.99148662302368951, 0.93009236359960523, 0.99156815147197164, 0.95335181301863836, 0.99268212590804139]

cq_coef1s=	[0.9931227431016576, 0.92050514502487424, 0.99280185681036059, 0.93679959697222281, 0.95187378315358151, 0.9914155558302391, 0.95161494535699631, 0.99236672132565751, 0.93126649344386114, 0.99310103557834384, 0.94237015193846396, 0.99194374346951064, 0.93894419975756449, 0.99194422856514086, 0.95590620862621356, 0.99291971159723746]

eta_coefs=	[]

cq0s=	[-6.2222, -0.7246, -6.0836, -0.7865, -0.7865, -6.0836, -0.7247, -6.2222, -0.7865, -6.0836, -0.7246, -6.2222, -0.7865, -6.0836, -0.7247, -6.2222]

cqprime0s=	[-6.1789057936366376, -0.65938545522402203, -6.0394629436308973, -0.73220529858590289, -0.74145683724337907, -6.0274439004737861, -0.69106026644329044, -6.172183005419325, -0.7238957922546041, -6.0378336609238135, -0.68312632069555623, -6.1692280657780012, -0.73151764397108954, -6.0323040062948863, -0.6908940588946072, -6.176666723825015]

cqprime1s=	[-6.1794083321271343, -0.66699802808502384, -6.039809376091509, -0.7367928830186532, -0.74864873045029179, -6.0313756754488423, -0.68963535090021522, -6.1747042134325065, -0.73244109709359673, -6.0416294600444118, -0.68284141209461102, -6.1720723606159886, -0.73847961310932442, -6.0345919088988902, -0.69274522939141703, -6.1781450295003308]
 
eta0s=	[0.50816, 0.61822, 0.47317, 0.58515, 0.58516, 0.47317, 0.61817, 0.50816, 0.58515, 0.47317, 0.61822, 0.50816, 0.58516, 0.47317, 0.61817, 0.50816]

etaprimes=	[]

fq0s=  [np.array([-1.56993638, -3.84921115, -5.41914754]),np.array([-0.20382264, -0.39262777, -0.59645041]),np.array([-1.42884634, -3.81517404, -5.24402038]),np.array([-0.21422497, -0.44204149, -0.65626646]),np.array([-0.21693544, -0.44762491, -0.66456035]),np.array([-1.42600282, -3.80758152, -5.23358433]),np.array([-0.21359636, -0.41149702, -0.62509338]),np.array([-1.56822826, -3.84502313, -5.41325138]),np.array([-0.21179381, -0.43702494, -0.64881875]),np.array([-1.42846088, -3.81414481, -5.24260568]),np.array([-0.21116118, -0.40676415, -0.61792533]),np.array([-1.56747747, -3.84318232, -5.41065978]),np.array([-0.21402743, -0.44162452, -0.65565195]),np.array([-1.42715264, -3.81065168, -5.23780433]),np.array([-0.21354499, -0.41139805, -0.62494304]),np.array([-1.56936748, -3.8478163 , -5.41718378])]

fqprimes=	[]